김성훈 교수님의 DeepLearningZeroToAll 저장소에 있는 소스코드 중 하나

https://github.com/hunkim/DeepLearningZeroToAll/blob/master/tf2/tf2-10-5-mnist_nn_dropout.py

를 'model.compile'과 'model.fit' API를 사용하지 않고 커스텀 루프로 학습하도록 바꿔봤더니 수렴속도나 최종 결과물에 엄청난 차이가 있음을 발견하였다.

우선 동일한 결과를 보장하기 위해 기본 세팅은 통일한다.

In [24]:
import tensorflow as tf
import random
from tensorflow.keras.datasets.mnist import load_data

random.seed(777)
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_rate = 0.3

데이터(MNIST) 준비

In [25]:
(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train.reshape(x_train.shape[0], 28 * 28)
x_test = x_test.reshape(x_test.shape[0], 28 * 28)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

model.compile과 fit을 사용하는 루프

In [26]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(input_dim=784, units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=10, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='softmax'))

model.compile(loss='categorical_crossentropy',
                 optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)

Epoch 1/15
600/600 [==============================] - 14s 23ms/step - loss: 3.2072 - accuracy: 0.7369
Epoch 2/15
600/600 [==============================] - 14s 23ms/step - loss: 0.4223 - accuracy: 0.8828
Epoch 3/15
600/600 [==============================] - 14s 24ms/step - loss: 0.3021 - accuracy: 0.9151
Epoch 4/15
600/600 [==============================] - 14s 23ms/step - loss: 0.2480 - accuracy: 0.9300
Epoch 5/15
600/600 [==============================] - 14s 23ms/step - loss: 0.2185 - accuracy: 0.9395
Epoch 6/15
600/600 [==============================] - 14s 23ms/step - loss: 0.2006 - accuracy: 0.9451
Epoch 7/15
600/600 [==============================] - 14s 23ms/step - loss: 0.1836 - accuracy: 0.9507
Epoch 8/15
600/600 [==============================] - 14s 23ms/step - loss: 0.1872 - accuracy: 0.9503
Epoch 9/15
600/600 [==============================] - 14s 23ms/step - loss: 0.1758 - accuracy: 0.9543
Epoch 10/15
600/600 [==============================] - 15s 24ms/step - loss: 0.184

학습결과를 확인해본다.

In [27]:
evaluation = model.evaluate(x_test, y_test)
print('loss: ', evaluation[0])
print('accuracy', evaluation[1])

313/313 [==============================] - 2s 5ms/step - loss: 0.1422 - accuracy: 0.9715
loss:  0.14218217134475708
accuracy 0.9714999794960022


이제 이를 커스텀 루프로 바꾼 뒤 다시 학습시켜본다. 

데이터로더를 커스텀 루프에 맞게 정의한다.

In [28]:
(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train.reshape(x_train.shape[0], 28 * 28)
x_test = x_test.reshape(x_test.shape[0], 28 * 28)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

data_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

model.compile과 fit을 사용하지 않는 커스텀 루프

In [29]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(input_dim=784, units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=10, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='softmax'))

criterion = lambda x, y : tf.keras.backend.mean(tf.keras.losses.categorical_crossentropy(x, y, from_logits=False))

optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(x_train) // batch_size

  for i, (batch_xs, batch_ys) in enumerate(data_train): 
    with tf.GradientTape() as tape:
      hypothesis = model(batch_xs, training=True)
      cost = tf.keras.backend.mean(criterion(batch_ys, hypothesis)) 
    grads = tape.gradient(cost, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    avg_cost += cost / total_batch

  print("[Epoch: %7d] cost = %5.5f"%(epoch + 1, avg_cost))

[Epoch:       1] cost = 12.92622
[Epoch:       2] cost = 13.92229
[Epoch:       3] cost = 14.30269
[Epoch:       4] cost = 14.30322
[Epoch:       5] cost = 14.30376
[Epoch:       6] cost = 14.30323
[Epoch:       7] cost = 14.00320
[Epoch:       8] cost = 14.08859
[Epoch:       9] cost = 13.94928
[Epoch:      10] cost = 12.82328
[Epoch:      11] cost = 12.81979
[Epoch:      12] cost = 12.81952
[Epoch:      13] cost = 12.82301
[Epoch:      14] cost = 12.82463
[Epoch:      15] cost = 12.82113


학습결과를 확인해본다.

In [30]:
hypothesis = model(x_test, training=False)
correct_prediction = tf.keras.backend.equal(tf.keras.backend.argmax(hypothesis, 1), tf.keras.backend.argmax(y_test, 1))
accuracy = tf.keras.backend.mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', accuracy.numpy())

Accuracy: 0.2092


엄청난 차이가 있음을 알 수 있다.

대체 무슨 차이가 있을까 내부를 뜯어봤더니

**로짓을 입력으로 받는 cross_entropy_with_logit 계열 함수들이 확률을 입력으로 받는 cross_entropy 계열 함수들보다 수치적으로 안정적(numerically unstable)이라는 사실을 알아냈다.**

아래는 함수들이 확률을 입력으로 받는 cross_entropy 계열 함수의 수식이다.

# -sum(label * log(probability))

이 수식의 경우 probability가 0일 경우에 log(0)이 되서 NaN이 뜰 확률이 높다.

근데 로짓을 입력으로 받는 cross_entropy_with_logit 계열 함수들은 이렇게 생겼고

# -sum(label * log(exp(logit) / sum(exp(logit))))

log(0)를 근본적으로 피할 수 있게 아래와 같이 reform이 가능하다.

# = -sum(label * log(exp(logit) / sum(exp(logit))))
# = -sum(label * (log(exp(logit)) - log(sum(exp(logit)))))
# = -sum(label * (logit - log(sum(exp(logit)))))

실제로 텐서플로우 API가 이런식으로 구현되어 있다.
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/kernels/xent_op.h?fbclid=IwAR0WBkHGDgz9NfzuwbXyza68djTouOxaCraG1jfXDeh8MetZ3InvVbkGXos#L35

따라서 마지막 레이어의 activation을 'softmax'대신 None으로 놓고

tf.keras.losses.categorical_crossentropy의 from_logits 인자를 True로 놓으면 안정적인 학습이 가능함을 알 수 있다.

In [31]:
(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train.reshape(x_train.shape[0], 28 * 28)
x_test = x_test.reshape(x_test.shape[0], 28 * 28)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

data_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(input_dim=784, units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
##############################################################바뀐 부분#####################################################################
model.add(tf.keras.layers.Dense(units=10, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation=None))

criterion = lambda x, y : tf.keras.backend.mean(tf.keras.losses.categorical_crossentropy(x, y, from_logits=True))
############################################################################################################################################

optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(x_train) // batch_size

  for i, (batch_xs, batch_ys) in enumerate(data_train): 
    with tf.GradientTape() as tape:
      hypothesis = model(batch_xs, training=True)
      cost = tf.keras.backend.mean(criterion(batch_ys, hypothesis)) 
    grads = tape.gradient(cost, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    avg_cost += cost / total_batch

  print("[Epoch: %7d] cost = %5.5f"%(epoch + 1, avg_cost))

[Epoch:       1] cost = 3.10352
[Epoch:       2] cost = 0.43175
[Epoch:       3] cost = 0.31126
[Epoch:       4] cost = 0.25589
[Epoch:       5] cost = 0.22204
[Epoch:       6] cost = 0.20632
[Epoch:       7] cost = 0.19979
[Epoch:       8] cost = 0.18638
[Epoch:       9] cost = 0.18818
[Epoch:      10] cost = 0.18887
[Epoch:      11] cost = 0.18071
[Epoch:      12] cost = 0.18631
[Epoch:      13] cost = 0.17571
[Epoch:      14] cost = 0.17700
[Epoch:      15] cost = 0.16782


안정적으로 학습되는 모습을 확인했으면 정확도도 확인해보자

In [32]:
hypothesis = model(x_test, training=False)
correct_prediction = tf.keras.backend.equal(tf.keras.backend.argmax(hypothesis, 1), tf.keras.backend.argmax(y_test, 1))
accuracy = tf.keras.backend.mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', accuracy.numpy())

Accuracy: 0.9701


'model.compile', 'model.fit'을 이용한 코드와 거의 동일한 성능을 냄을 알 수 있다.

그렇다면 왜 'model.compile', 'model.fit'을 이용한 코드는 확률을 입력으로 받는 cross_entropy 계열 함수를 써도 안정적인 학습이 가능할까?

아마도 model.compile 함수 내부에서 전체 그래프를 컴파일하는 과정에서 이러한 부분을 자동으로 Reformulation 해주는 것으로 보인다.

실제로 'model.compile', 'model.fit'을 이용한 코드의 마지막 레이어의 activation을 'softmax'대신 None으로 놓고,

로스 함수를 키워드로 불러오지 않고(키워드로 불러오면 from_logits 인자가 False로 설정됨) tf.keras.losses.categorical_crossentropy로 불러오면서 from_logits 인자를 True로 놓으면 원본과 학습결과가 별로 달라지지 않음을 알 수 있다.

In [33]:
(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train.reshape(x_train.shape[0], 28 * 28)
x_test = x_test.reshape(x_test.shape[0], 28 * 28)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(input_dim=784, units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_rate))
model.add(tf.keras.layers.Dense(units=10, kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.05), use_bias=True, activation=None))

model.compile(loss=lambda x, y : tf.keras.backend.mean(tf.keras.losses.categorical_crossentropy(x, y, from_logits=True)) ,
                 optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)

evaluation = model.evaluate(x_test, y_test)
print('loss: ', evaluation[0])
print('accuracy', evaluation[1])

Epoch 1/15
600/600 [==============================] - 14s 24ms/step - loss: 3.1609 - accuracy: 0.7393
Epoch 2/15
600/600 [==============================] - 13s 22ms/step - loss: 0.4348 - accuracy: 0.8783
Epoch 3/15
600/600 [==============================] - 14s 24ms/step - loss: 0.3077 - accuracy: 0.9131
Epoch 4/15
600/600 [==============================] - 14s 23ms/step - loss: 0.2515 - accuracy: 0.9297
Epoch 5/15
600/600 [==============================] - 15s 24ms/step - loss: 0.2133 - accuracy: 0.9407
Epoch 6/15
600/600 [==============================] - 14s 23ms/step - loss: 0.1957 - accuracy: 0.9464
Epoch 7/15
600/600 [==============================] - 13s 22ms/step - loss: 0.1973 - accuracy: 0.9473
Epoch 8/15
600/600 [==============================] - 13s 22ms/step - loss: 0.1853 - accuracy: 0.9505
Epoch 9/15
600/600 [==============================] - 13s 22ms/step - loss: 0.1825 - accuracy: 0.9533
Epoch 10/15
600/600 [==============================] - 14s 24ms/step - loss: 0.184

정리하자면

1. 로짓을 입력으로 받는 cross_entropy_with_logit 계열 함수들이 확률을 입력으로 받는 cross_entropy 계열 함수들보다 수치적으로 안정적이다.
2. model.compile API는 둘 중 뭘 써도 알아서 안정적으로 바꿔준다.
3. 하지만 커스텀 루프는 사용자가 코딩해놓은 대로 정직하게(?) 동작하기 때문에 마지막 레이어의 activation을 softmax대신 None으로 해놓고 cross_entropy_with_logit을 사용해야 안정적으로 작동한다.